# Install Packages dibawah terlebih dahulu

In [7]:
import pyderman as driver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import numpy as np
import time
import os
import re
driver = webdriver.Firefox()


Masuk ke website evisum - Masukkan USERNAME DAN PASSWORD SERTA SELESAIKAN CAPTCHA

![Halaman Evsium](img/evisum.png)

In [8]:
pkb = pd.read_excel("data/pkb_kec.xlsx")
pkb['NIP'] = pkb['NIP'].astype(str)
pkb = pkb[['Kabupaten', 'Kecamatan', 'NIP', 'Nama']]
pkb

,Kabupaten,Kecamatan,NIP,Nama
0,PASANGKAYU,BAMBALAMOTU,196911152006042008,KUR`ANI
1,PASANGKAYU,BAMBALAMOTU,199702082023212029,"FITRI DAYANTI, S.E"
2,PASANGKAYU,BAMBALAMOTU,197009292012122002,NUR AYNY
3,PASANGKAYU,BAMBALAMOTU,199707072023212027,"SARI ASTUTI, S.Sos"
4,PASANGKAYU,BAMBALAMOTU,198512032011011007,"HASANUDDIN, SE"
...,...,...,...,...
422,MAMUJU TENGAH,KAROSSA,198702252022212002,"HANA PRATIWI ARSYAD, S.ST"
423,MAMUJU TENGAH,KAROSSA,199309132022212003,"IVA MUKRIMAH, S.Tr.Keb"
424,MAMUJU TENGAH,KAROSSA,198206202022211002,"ABDULLAH, A.Md.Kep"
425,MAMUJU TENGAH,KAROSSA,199501012023212066,"NURHASIA L., A.Md.Keb"


In [9]:
driver.get("https://evisum4.bkkbn.go.id/")

In [10]:
# Tentukan bulan dan tahun yang diinginkan
year = 2024
month = 7

# Generate semua tanggal pada bulan tertentu
date_range = pd.date_range(start=f'{year}-{month:02d}-01', end=f'{year}-{month:02d}-31')

# Filter untuk menghapus hari Sabtu (5) dan Minggu (6)
weekdays = date_range[~date_range.weekday.isin([5, 6])]

# Format tanggal menjadi dd-mm-yyyy
formatted_dates = weekdays.strftime('%d-%m-%Y')
print(formatted_dates)

Index(['01-07-2024', '02-07-2024', '03-07-2024', '04-07-2024', '05-07-2024',
       '08-07-2024', '09-07-2024', '10-07-2024', '11-07-2024', '12-07-2024',
       '15-07-2024', '16-07-2024', '17-07-2024', '18-07-2024', '19-07-2024',
       '22-07-2024', '23-07-2024', '24-07-2024', '25-07-2024', '26-07-2024',
       '29-07-2024', '30-07-2024', '31-07-2024'],
      dtype='object')


In [24]:
#formatted_dates = formatted_dates[0:11] #mengambil tanggal 1-16

In [11]:
#membuat dataframe kosong
presensi_pkb = pd.DataFrame(columns = ['Kabupaten', 'Kecamatan', 'NIP', 'Nama',
                               'Tanggal', 'lat1', 'lon1', 'lat2', 'lon2'])
presensi_pkb

,Kabupaten,Kecamatan,NIP,Nama,Tanggal,lat1,lon1,lat2,lon2


In [12]:
for i in range(0, 427): #jumlah PKB
    for j in range(0, len(formatted_dates)): #jumlah hari
        print(str(i), " - ", str(j))

        link = "https://evisum4.bkkbn.go.id/showMapPresensi?nip=" + pkb.iloc[i, 2] + "&tgl=" + formatted_dates[j]

        driver.get(link)
        sleep(0.5)

        script_element = driver.find_element(By.XPATH, "/html/body/script")

        # Mengambil teks dari elemen <script>
        script_text = script_element.get_attribute('innerHTML')

        # Regex untuk mengekstrak koordinat dan URL gambar
        coordinates = re.findall(r'L\.marker\(\[(.*?)\]\)', script_text)

        if len(coordinates) > 1:
            coordinates_datang = coordinates[0]
            lat1, lon1 = coordinates_datang.split(', ')

            cordinates_pulang = coordinates[1]
            lat2, lon2 = cordinates_pulang.split(', ')
        elif len(coordinates) == 1:
            coordinates_datang = coordinates[0]
            lat1, lon1 = coordinates_datang.split(', ')
            lat2, lon2 = [0, 0]
        else:
            lat1, lon1, lat2, lon2 = [0, 0, 0, 0]

        # Memisahkan latitude dan longitude
        # if len(coordinates[0]) > 0:
        #     coordinates_datang = coordinates[0]
        #     lat1, lon1 = coordinates[0].split(', ')
        # else:
        #     lat1 = 0
        #     lon1 = 0

        # # Memisahkan latitude dan longitude
        # if len(coordinates[1]) > 0:
        #     cordinates_pulang = coordinates[1]
        #     lat2, lon2 = coordinates[1].split(', ')
        # else:
        #     lat2 = 0
        #     lon2 = 0

        presensi_pkb_sementara = pd.DataFrame({'Kabupaten': pkb.iloc[i,0],
                    'Kecamatan': pkb.iloc[i,1],
                    'NIP': pkb.iloc[i,2],
                    'Nama': pkb.iloc[i,3],
                    'Tanggal': formatted_dates[j],
                    'lat1': lat1,
                    'lon1': lon1,
                    'lat2': lat2,
                    'lon2': lon2}, index=[0])
        
        presensi_pkb = pd.concat([presensi_pkb, presensi_pkb_sementara], ignore_index=True)
        print('Berhasil')
print("Selesai")
        

0  -  0
Berhasil
0  -  1
Berhasil
0  -  2
Berhasil
0  -  3
Berhasil
0  -  4
Berhasil
0  -  5
Berhasil
0  -  6
Berhasil
0  -  7
Berhasil
0  -  8
Berhasil
0  -  9
Berhasil
0  -  10
Berhasil
0  -  11
Berhasil
0  -  12
Berhasil
0  -  13
Berhasil
0  -  14
Berhasil
0  -  15
Berhasil
0  -  16
Berhasil
0  -  17
Berhasil
0  -  18
Berhasil
0  -  19
Berhasil
0  -  20
Berhasil
0  -  21
Berhasil
0  -  22
Berhasil
1  -  0
Berhasil
1  -  1
Berhasil
1  -  2
Berhasil
1  -  3
Berhasil
1  -  4
Berhasil
1  -  5
Berhasil
1  -  6
Berhasil
1  -  7
Berhasil
1  -  8
Berhasil
1  -  9
Berhasil
1  -  10
Berhasil
1  -  11
Berhasil
1  -  12
Berhasil
1  -  13
Berhasil
1  -  14
Berhasil
1  -  15
Berhasil
1  -  16
Berhasil
1  -  17
Berhasil
1  -  18
Berhasil
1  -  19
Berhasil
1  -  20
Berhasil
1  -  21
Berhasil
1  -  22
Berhasil
2  -  0
Berhasil
2  -  1
Berhasil
2  -  2
Berhasil
2  -  3
Berhasil
2  -  4
Berhasil
2  -  5
Berhasil
2  -  6
Berhasil
2  -  7
Berhasil
2  -  8
Berhasil
2  -  9
Berhasil
2  -  10
Berhasil
2  -

In [10]:
presensi_pkb

,Kabupaten,Kecamatan,NIP,Nama,Tanggal,lat1,lon1,lat2,lon2
0,PASANGKAYU,BAMBALAMOTU,196911152006042008,KUR`ANI,01-08-2024,-1.1753366666667,119.37623833333,-1.174635,119.376505
1,PASANGKAYU,BAMBALAMOTU,196911152006042008,KUR`ANI,02-08-2024,-1.17458666666667,119.376376666667,-1.1745957,119.3767491
2,PASANGKAYU,BAMBALAMOTU,196911152006042008,KUR`ANI,05-08-2024,-2.6722541,118.8908479,0,0
3,PASANGKAYU,BAMBALAMOTU,196911152006042008,KUR`ANI,06-08-2024,-1.1762867,119.3695666,-1.1741861,119.3756639
4,PASANGKAYU,BAMBALAMOTU,196911152006042008,KUR`ANI,07-08-2024,-1.1746316666667,119.37655833333,-1.1764816666667,119.369485
...,...,...,...,...,...,...,...,...,...
9389,MAMUJU TENGAH,KAROSSA,197206011994031006,"NASARANI, AMD.Kep.",26-08-2024,-1.7928512,119.4061419,-1.7837566666667,119.39036166667
9390,MAMUJU TENGAH,KAROSSA,197206011994031006,"NASARANI, AMD.Kep.",27-08-2024,-1.7839183333333,119.39009,-1.78358,119.390271666667
9391,MAMUJU TENGAH,KAROSSA,197206011994031006,"NASARANI, AMD.Kep.",28-08-2024,-1.7866116666667,119.392865,-1.78363,119.39033833333
9392,MAMUJU TENGAH,KAROSSA,197206011994031006,"NASARANI, AMD.Kep.",29-08-2024,-1.7838683333333,119.390135,-1.783755,119.390505


In [11]:
presensi_pkb.to_excel("data/agustus_presensi_full.xlsx")

In [ ]:
# batas bawah